<a href="https://colab.research.google.com/github/abhirockraj/pytorch_nn/blob/main/Finacial_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import pytorch_lightning as pl
from pytorch_lightning import Trainer

In [8]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [11]:
adni = yf.Ticker("ADANIENT.NS")

In [12]:
adni.financials

,2021-03-31,2020-03-31,2019-03-31,2018-03-31
Research Development,None,None,None,None
Effect Of Accounting Charges,None,None,None,None
Income Before Tax,1.38541e+10,1.36432e+10,5.5967e+09,6.587e+09
Minority Interest,1.75144e+10,1.26337e+10,3.8777e+09,7.7786e+09
Net Income,9.2264e+09,1.13817e+10,7.1714e+09,7.5725e+09
Selling General Administrative,6.78426e+10,6.45792e+10,5.08845e+10,3.89138e+10
Gross Profit,9.28931e+10,8.74223e+10,7.05725e+10,5.92558e+10
Ebit,1.98188e+10,1.85189e+10,1.59071e+10,1.38368e+10
Operating Income,1.98188e+10,1.85189e+10,1.59071e+10,1.38368e+10
Other Operating Expenses,-1.397e+08,-3.964e+08,-1.168e+08,-1.34e+08


In [13]:
hist = adni.history(period="5y")

In [14]:
hist.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2016-10-04,36.072186,36.713472,35.804985,36.339390,4761110,0.0,0
2016-10-05,36.553150,36.766911,35.858428,36.152348,2904432,0.0,0
2016-10-06,36.339385,36.633305,35.350740,35.671383,2998323,0.0,0
2016-10-07,35.885149,36.526432,35.430903,36.259228,5426409,0.0,0
2016-10-10,36.526434,37.595239,35.858431,36.900513,9051818,0.0,0


In [18]:
df_x_close_out = hist[['Open','High','Low']]
df_y_close_out = hist['Close']

In [21]:
df_x_close_out.reset_index(drop=True, inplace=True)
df_y_close_out.reset_index(drop=True, inplace=True)

In [28]:
df_x_close_out_val= df_x_close_out[:247]
df_y_close_out_val = df_y_close_out[:247]
df_x_close_out_train= df_x_close_out[247:]
df_y_close_out_train = df_y_close_out[247:]

Pytorch model creation

In [37]:
input_size =3
hidden_size=3
lr= 0.001
no_of_epoch=100
batch_size=100

In [38]:
class NeuralNet(pl.LightningModule):
    def __init__(self,input_size=3,hidden_size=3,num_layer=50):
        super(NeuralNet,self).__init__()
        self.rnn1= nn.RNN(input_size,hidden_size,num_layers,batch_first=True)
        self.linear1=nn.Linear(hidden_size,1)
    def forward(self,x):
        out=self.rnn1(x)
        out=self.linear2(out)
        return out
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=lr)
        return optimizer
    def training_step(self, train_batch, batch_idx):
        x, y = train_batch
        y_hat = self(x)
        loss = F.mse_loss(y_hat, y)
        self.log('train_loss', loss)
        return loss
    def validation_step(self, val_batch, batch_idx):
        x, y = val_batch
        y_hat = self(x)
        loss = F.mse_loss(y_hat, y)
        self.log('val_loss', loss)

In [44]:
df_x_close_out_val =torch.Tensor(df_x_close_out_val.values)
df_y_close_out_val =torch.Tensor(df_y_close_out_val.values)
df_x_close_out_train= torch.Tensor(df_x_close_out_train.values)
df_y_close_out_train= torch.Tensor(df_y_close_out_train.values)

In [ ]:
loader = DataLoader((df_x_close_out_train,df_y_close_out_train),batch_size =batch_size,num_worker=4,shuffle=True)